# Claude Code Colab Bootstrap

**What this notebook does:**
- Installs Claude Code in your Colab environment
- Configures the claude-colab plugin from the claude-colab marketplace
- Sets up skills, agents, hooks, and commands automatically

**How to use it:**
1. **Get your auth token** (see cell below) and save it to Colab Secrets
2. Run all cells → open terminal → run `claude`
3. The plugin provides Colab-specific skills and agents automatically

> **Ephemeral by default:** No Google Drive access needed. Just re-run the notebook each session.
> Check `USE_GOOGLE_DRIVE` below if you want persistence between sessions.

---

## 🔑 Authentication Setup

**To get a long-lived Claude Code OAuth token:**

1. On your local machine (laptop/desktop), run:
   ```bash
   claude setup-token
   ```
2. Copy the token that's printed
3. In Colab, click the **key icon** (🔑) in the left sidebar
4. Click **"Add secret"**
5. Name: `CLAUDE_CODE_OAUTH_TOKEN`
6. Value: Paste your token
7. Click **"Add"**

To verify, run this in a Python cell:
```python
from google.colab import userdata
userdata.get("CLAUDE_CODE_OAUTH_TOKEN")
```

The notebook will automatically use this token to authenticate Claude Code.

---

In [ ]:
# @title 1. Project Settings { display-mode: "form" }

# @markdown ### Storage Mode
USE_GOOGLE_DRIVE = False  # @param {type:"boolean"}
# @markdown - **Unchecked (default):** Ephemeral — no Drive access, resets each session
# @markdown - **Checked:** Persistent — saves to Google Drive

# @markdown ---
# @markdown ### Project
PROJECT_NAME = "my-project"  # @param {type:"string"}
PROJECT_TYPE = "ML/AI Development"  # @param ["ML/AI Development", "Data Science/Analysis", "Web Development", "General Python", "Custom"]
PROJECT_DESCRIPTION = ""  # @param {type:"string"}

# @markdown ---
# @markdown ### Plugin Version
BOOTSTRAP_VERSION = "{{BOOTSTRAP_VERSION}}"  # Version baked into notebook at build time

# Derived paths
if USE_GOOGLE_DRIVE:
    WORKSPACE_PATH = f"/content/drive/My Drive/claude-workspaces/{PROJECT_NAME}"
    STORAGE_MODE = "persistent (Google Drive)"
else:
    WORKSPACE_PATH = f"/content/claude-workspaces/{PROJECT_NAME}"
    STORAGE_MODE = "ephemeral (resets each session)"

CLAUDE_CONFIG_PATH = f"{WORKSPACE_PATH}/.claude"

print(f"Project: {PROJECT_NAME}")
print(f"Storage: {STORAGE_MODE}")
print(f"Workspace: {WORKSPACE_PATH}")
print(f"Bootstrap version: {BOOTSTRAP_VERSION}")

In [ ]:
# @title 2. Mount Google Drive (if enabled)

if USE_GOOGLE_DRIVE:
    from google.colab import drive

    print("Mounting Google Drive...")
    drive.mount("/content/drive")
    print("✓ Drive mounted")
else:
    print("⏭️ Skipping Drive mount (ephemeral mode)")

In [ ]:
# @title 3. Install Claude Code & Dependencies
import json as json_lib
import os
import shutil
import subprocess
import sys

print("=" * 50)
print("Installing dependencies...")
print("=" * 50)

# Sandbox deps
print("\n[1/3] Sandbox dependencies...")
subprocess.run(
    "apt-get update -qq && apt-get install -qq -y socat bubblewrap > /dev/null 2>&1", shell=True
)
print("  ✓ socat, bubblewrap")

# Claude Code
print("\n[2/3] Claude Code...")
result = subprocess.run(
    "curl -fsSL https://claude.ai/install.sh | bash", shell=True, capture_output=True, text=True
)
if result.returncode == 0:
    print("  ✓ Claude Code installed")
else:
    print(f"  ⚠️ Install issue: {result.stderr[:200]}")
    print("  Retrying...")
    subprocess.run("curl -fsSL https://claude.ai/install.sh | bash", shell=True)

# Configure environment
print("\n[3/3] Configuring environment...")
bashrc_path = os.path.expanduser("~/.bashrc")
with open(bashrc_path, "r") as f:
    bashrc_content = f.read()
if ".local/bin" not in bashrc_content:
    with open(bashrc_path, "a") as f:
        f.write("\n# Claude Code\n")
        f.write('export PATH="$HOME/.local/bin:$HOME/.claude/bin:$PATH"\n')
        f.write("export TERM=xterm-256color\n")
        f.write("export COLORTERM=truecolor\n")
        f.write("export FORCE_COLOR=1\n")
print("  ✓ PATH and terminal configured")

# Create workspace
os.makedirs(WORKSPACE_PATH, exist_ok=True)
os.makedirs(CLAUDE_CONFIG_PATH, exist_ok=True)
print(f"  ✓ Workspace: {WORKSPACE_PATH}")

# Symlink ~/.claude
home_claude = os.path.expanduser("~/.claude")
if os.path.islink(home_claude):
    os.unlink(home_claude)
elif os.path.exists(home_claude):
    if os.path.isdir(home_claude):
        shutil.rmtree(home_claude)
    else:
        os.unlink(home_claude)
os.symlink(CLAUDE_CONFIG_PATH, home_claude)
print(f"  ✓ ~/.claude → {CLAUDE_CONFIG_PATH}")

print("\n" + "=" * 50)
print("✅ Installation complete!")
print("=" * 50)

In [ ]:
# @title 4. Configure Authentication

print("Configuring authentication...")

# Try to get token from Colab Secrets
oauth_token = None
try:
    from google.colab import userdata

    try:
        oauth_token = userdata.get("CLAUDE_CODE_OAUTH_TOKEN")
        if oauth_token:
            print("  ✓ Found CLAUDE_CODE_OAUTH_TOKEN in Colab Secrets")
    except Exception as e:
        print(f"  ⚠️ Could not read CLAUDE_CODE_OAUTH_TOKEN: {e}")

    # Backwards compatibility
    if not oauth_token:
        try:
            oauth_token = userdata.get("CLAUDE_CODE_TOKEN")
            if oauth_token:
                print("  ✓ Found CLAUDE_CODE_TOKEN in Colab Secrets")
                print("  ⚠️ Consider renaming it to CLAUDE_CODE_OAUTH_TOKEN")
        except Exception as e:
            print(f"  ⚠️ Could not read CLAUDE_CODE_TOKEN: {e}")
except ImportError:
    print("  ⚠️ Not in Colab environment")

# Also check environment variable
if not oauth_token:
    oauth_token = os.environ.get("CLAUDE_CODE_OAUTH_TOKEN")
    if oauth_token:
        print("  ✓ Found CLAUDE_CODE_OAUTH_TOKEN in environment")

if oauth_token:
    # 1. Export token to bashrc so it's available in terminal sessions
    bashrc_path = os.path.expanduser("~/.bashrc")
    with open(bashrc_path, "a") as f:
        f.write(f'\nexport CLAUDE_CODE_OAUTH_TOKEN="{oauth_token}"\n')
    print("  ✓ Exported CLAUDE_CODE_OAUTH_TOKEN to ~/.bashrc")

    # 2. Create ~/.claude.json with hasCompletedOnboarding to skip onboarding wizard
    claude_json_path = os.path.expanduser("~/.claude.json")
    claude_json = {"hasCompletedOnboarding": True}
    if os.path.exists(claude_json_path):
        try:
            with open(claude_json_path, "r") as f:
                existing = json_lib.load(f)
            existing["hasCompletedOnboarding"] = True
            claude_json = existing
        except Exception:
            pass
    with open(claude_json_path, "w") as f:
        json_lib.dump(claude_json, f, indent=2)
    print("  ✓ Created ~/.claude.json with hasCompletedOnboarding=true")

    print("  ✓ Authentication configured!")
else:
    print("\n" + "!" * 60)
    print("⚠️  No Claude Code OAuth token found")
    print("Add this Colab Secret: CLAUDE_CODE_OAUTH_TOKEN")
    print("Generate it with: claude setup-token")
    print("Verify in a Python cell:")
    print("from google.colab import userdata")
    print("userdata.get('CLAUDE_CODE_OAUTH_TOKEN')")
    print("!" * 60 + "\n")
    print("📝 See the authentication setup cell above for instructions.")
    print("💡 You can also authenticate manually in the terminal with: claude")

In [ ]:
# @title 5. Configure Plugin Marketplace
import json as json_lib

print("Configuring plugin marketplace...")

# Create/update settings.json with marketplace and plugin config
settings_path = f"{CLAUDE_CONFIG_PATH}/settings.json"
settings = {}
if os.path.exists(settings_path):
    with open(settings_path, "r") as f:
        settings = json_lib.load(f)

# Add marketplace configuration
# This tells Claude Code where to find the claude-colab plugin
settings["extraKnownMarketplaces"] = {
    "claude-colab": {
        "source": {
            "source": "github",
            "repo": "{{GITHUB_REPO}}"
        }
    }
}

# Enable the claude-colab plugin
settings["enabledPlugins"] = ["claude-colab@claude-colab"]

# Sensible defaults for Colab
settings.setdefault("permissions", {})
settings["permissions"].setdefault("allow", [])
settings["permissions"]["allow"].extend([
    "Bash(pip:*)",
    "Bash(python:*)",
    "Bash(python3:*)",
    "Read(*)",
    "Write(/content/**)"
])
# Deduplicate
settings["permissions"]["allow"] = list(set(settings["permissions"]["allow"]))

with open(settings_path, "w") as f:
    json_lib.dump(settings, f, indent=2)

print("  ✓ Marketplace: claude-colab (github.com/{{GITHUB_REPO}})")
print("  ✓ Plugin enabled: claude-colab")
print("  ✓ Permissions configured for Colab")
print(f"  ✓ Settings saved to {settings_path}")

In [ ]:
# @title 6. Capture Environment
import json as json_lib

print("Capturing environment snapshot...")

env_snapshot = {
    "storage_mode": "persistent" if USE_GOOGLE_DRIVE else "ephemeral",
    "workspace_path": WORKSPACE_PATH,
    "project_name": PROJECT_NAME,
    "project_type": PROJECT_TYPE,
    "python_version": sys.version.split()[0],
    "bootstrap_version": BOOTSTRAP_VERSION,
}

# GPU
try:
    import torch

    env_snapshot["gpu_available"] = torch.cuda.is_available()
    env_snapshot["pytorch_version"] = torch.__version__
    if torch.cuda.is_available():
        env_snapshot["gpu_name"] = torch.cuda.get_device_name(0)
        env_snapshot["gpu_memory_gb"] = round(
            torch.cuda.get_device_properties(0).total_memory / 1e9, 1
        )
        print(f"  GPU: {env_snapshot['gpu_name']} ({env_snapshot['gpu_memory_gb']}GB)")
    else:
        print("  ⚠️ No GPU (Runtime → Change runtime type → GPU)")
except Exception:
    env_snapshot["gpu_available"] = False

# Drive
drive_mounted = os.path.exists("/content/drive/My Drive") or os.path.exists(
    "/content/drive/MyDrive"
)
env_snapshot["drive_mounted"] = drive_mounted
if USE_GOOGLE_DRIVE and not drive_mounted:
    print("  ⚠️ Drive not mounted!")

# Tools
env_snapshot["tools"] = {}
for tool in ["claude", "git", "socat", "bwrap"]:
    r = subprocess.run(f"which {tool}", shell=True, capture_output=True)
    env_snapshot["tools"][tool] = r.returncode == 0

# Write
with open(f"{WORKSPACE_PATH}/ENVIRONMENT.json", "w") as f:
    json_lib.dump(env_snapshot, f, indent=2)

print("\n✓ Saved to ENVIRONMENT.json")

In [ ]:
# @title 7. Generate Bootstrap Prompt { display-mode: "form" }

# @markdown ### What should Claude set up?
create_claude_md = True  # @param {type:"boolean"}
run_diagnostics = True  # @param {type:"boolean"}

# Build prompt
if USE_GOOGLE_DRIVE:
    storage_note = f"Workspace: {WORKSPACE_PATH} (persistent on Drive)"
else:
    storage_note = f"Workspace: {WORKSPACE_PATH} (ephemeral - resets each session)"

prompt_parts = [
    f"Hi Claude! I'm using Claude Code in Google Colab for {PROJECT_TYPE}.",
    "",
    f"**Environment:** {storage_note}",
    "",
    "The claude-colab plugin is already installed with skills, agents, and hooks.",
    "",
    "Please read ENVIRONMENT.json to understand my setup.",
    "",
    "**Please do:**",
]

if create_claude_md:
    prompt_parts.append(f"- Create a CLAUDE.md with {PROJECT_TYPE} conventions and Colab-specific notes")
if run_diagnostics:
    prompt_parts.append("- Run /colab-status to verify everything is working")

if PROJECT_DESCRIPTION:
    prompt_parts.append(f"\n**Project details:** {PROJECT_DESCRIPTION}")

prompt_parts.append("\nGive me a summary when done.")

BOOTSTRAP_PROMPT = "\n".join(prompt_parts)

print("=" * 60)
print("COPY THIS PROMPT INTO CLAUDE CODE:")
print("=" * 60)
print()
print(BOOTSTRAP_PROMPT)
print()
print("=" * 60)
print()
print("TERMINAL COMMANDS:")
print("-" * 40)
print("source ~/.bashrc")
print(f'cd "{WORKSPACE_PATH}"')
print("claude")

---

## Reference

### Plugin Features

The **claude-colab** plugin provides:

| Type | Name | Description |
|------|------|-------------|
| **Skill** | claude-expert | Claude Code reference and best practices |
| **Skill** | ipynb | Jupyter notebook manipulation |
| **Skill** | customize | Environment customization helper |
| **Agent** | colab | Colab environment expert |
| **Agent** | notebook-doctor | Diagnose and fix issues |
| **Command** | /colab-status | Check environment status |
| **Command** | /checkpoint | Save to Google Drive |
| **Command** | /colab-update | Check for plugin updates |
| **Hook** | SessionStart | Auto-checks for updates |

### Future Sessions

**Ephemeral mode:** Just re-run this notebook each session.

**Persistent mode:** After first setup, future sessions only need:
```python
from google.colab import drive
drive.mount('/content/drive')
!curl -fsSL https://claude.ai/install.sh | bash
# Then: source ~/.bashrc && cd /content/drive/My\ Drive/claude-workspaces/PROJECT && claude
```

### Useful Commands

| Command | Purpose |
|---------|---------|
| `/colab-status` | Check Colab environment |
| `/checkpoint` | Save to Drive |
| `/colab-update` | Check for updates |
| `/init` | Generate CLAUDE.md from codebase |
| `/clear` | Reset conversation context |
| `/compact` | Summarize to save tokens |
| `/model opus` | Switch to Opus |
| `/usage` | Check token usage |

> **Note:** `/doctor` may hang in Colab's terminal. Use `/colab-status` instead.